In [ ]:
# Add Path to Bitcoin Core Testframework Library.
import sys
sys.path.insert(0, '/Users/jamesc/Dropbox/repos/bitcoin_sipa/test/functional')

# 2. TapTweak

* 2.1 Tweaking the Public Key
* 2.2 Commitment Schemes with Tweaks
* 2.3 Script Commitments


## 2.1 Tweaking the Public Key

The script commitment scheme of taproot is based on tweaking the public key in the `Segwit version 1 output`. Tweaking a public key means to alter it with a value (the tweak) so that it remains spendable with knowledge of the original private key and tweak. Very importantly, an observer cannot distinguish between a tweaked and untweaked public key.

Tweaking can be easily achieved with the additive and multiplicative elliptic curve math operations.

![test](images/taptweak0.jpg)


### Example 2.1.1 : Additive Tweak - Signing with a tweaked keypair

* 1. Key pair generation: `xG = P`
* 2. A tweak is positive scalar value < curve order: `t < SECP256K1 Order` 
* 3. A tweak has a corresponding point: `T = t*G`
* 4. A private key is tweaked by the tweak scalar: `x' = x + t`
* 5. The public key can be tweaked by the tweak point: `P' = P + T`
* 6. Verify that signature with tweaked key pair is valid.

In [ ]:
from test_framework.key import ECKey, ECPubKey
import hashlib

# 1) 
privatekey = ECKey()
privatekey.generate()
publickey = privatekey.get_pubkey()

# 2) 
tweak = hashlib.sha256(b'tweak').digest()

# 3) 
tweak_private = ECKey()
tweak_private.set(tweak, True)
tweak_point = tweak_private.get_pubkey()

# 4) 
privatekey_tweaked = privatekey.add(tweak)

# 5)
publickey_tweaked = publickey.add(tweak_point)

# 6)
msg = hashlib.sha256(b'msg').digest()
sig = privatekey_tweaked.sign_schnorr(msg)
print(publickey_tweaked.verify_schnorr(sig, msg))

### Example 2.1.2: Additive Tweak - Signing with a tweaked Musig keys

* 1. Key pair generation for all participants: `xG = P`
* 2. Aggregate all public keys: `pk_musig`
* 3. Apply challenge factors to all private keys: `x' = c * x`
* 4. Tweak the `pk_musig`.
* 5. Signing -  Generate indidividual nonces & aggregate: `R_agg`
* 6. Signing - Sign individually & aggregate: `sig_agg`
    * **One signer must sign with tweaked key.**

In [ ]:
from test_framework.key import ECKey, ECPubKey
from test_framework.key import generate_musig_key
from test_framework.key import generate_schnorr_nonce, aggregate_schnorr_nonces
from test_framework.key import aggregate_musig_signatures

import random

# 1) 
keys = []
pubkeys = []
for _ in range(5):
    private_key = ECKey()
    private_key.generate()
    public_key = private_key.get_pubkey()
    keys.append((private_key, public_key))
    pubkeys.append(public_key)
    
# 2) 
c_map, pk_musig = generate_musig_key(pubkeys)

# 3) 
keys_c = []
for private, public in keys:
    private_c = private.mul(c_map[public])
    public_c = public.mul(c_map[public])
    keys_c.append((private_c, public_c))
    
# 4)
tweak = hashlib.sha256(b'tweak').digest()
pk_musig_tweaked = pk_musig.tweak_add(tweak)

# 5) 
nonce_map = {}
nonce_points = []
for private_c, public_c in keys_c:
    nonce_map[public_c] = generate_schnorr_nonce()
    nonce_points.append(nonce_map[public_c].get_pubkey())
R_agg, negated = aggregate_schnorr_nonces(nonce_points)

# 6) 
msg = hashlib.sha256(b'msg').digest()
sigs = []
for idx, (private_c, public_c) in enumerate(keys_c):
    # One person must tweak keys.
    private_c = private_c.add(tweak) if idx == 0 else private_c # One person must tweak secret.
    signature = private_c.sign_musig(nonce_map[public_c], negated, R_agg, pk_musig_tweaked, msg)
    sigs.append(signature)
sig_agg = aggregate_musig_signatures(sigs)

pk_musig_tweaked.verify_schnorr(sig_agg, msg)


## 2.2 Commitment Schemes with Tweaks

Taproot uses the tweak as a commitment for spending script paths. However, simply applying the committed value as a public key tweak is not sufficient: For a given public key point Q with a known discrete log, the untweaked private key can always be solved for given a tweak of any value. 


Instead, the committed value must first be hashed with the untweaked public key point. This prevents modification of both untweaked secret and tweak for a given pubkey point Q.

![test](images/taptweak1.jpg)



### Example 2.2.1 : Modify the Tweak/Secret for a given tweaked Public Key Q.

* 1. Tweaking the public key to obtain: `Point Q`
* 2. Create a new tweak for the same point Q: `t'`
* 3. Solve for `x'` so that `x'G + t'G = Q`

In [ ]:
from test_framework.key import ECKey, ECPubKey, SECP256K1_ORDER

# 1)
x = ECKey()
x.generate()
pk = x.get_pubkey()
t = hashlib.sha256(b'tweak').digest()
t_int = int.from_bytes(t, "big")
Q = pk.tweak_add(t)

# 2)
t2 = hashlib.sha256(b'tweak2').digest()
t2_int = int.from_bytes(t2, "big")

# 3) x` = x - (t' - t)
x_int = int.from_bytes(x.get_bytes(),"big")
s = (t2_int + (SECP256K1_ORDER - t_int)% SECP256K1_ORDER) % SECP256K1_ORDER
x2_int = (x_int + (SECP256K1_ORDER - s)% SECP256K1_ORDER) % SECP256K1_ORDER

print((x2_int + t2_int)% SECP256K1_ORDER == (x_int + t_int)% SECP256K1_ORDER)

### Example 2.2.2 - Tweaking Pubkey with `H(P|msg)`

* 1. Key pair generation: 
* 2. **A tweak is hash of both t and P: `H(P|msg) = t`**
* 3. A tweak has a corresponding point: `T = t*G`
* 4. A private key is tweaked by the tweak scalar: `x' = x + t`
* 5. The public key can be tweaked by the tweak point: `P' = P + T`
* 6. Verify that signature with tweaked key pair is valid.

In [ ]:
from test_framework.key import ECKey, ECPubKey
import hashlib

# 1) 
privatekey = ECKey()
privatekey.generate()
publickey = privatekey.get_pubkey()

# 2) 
# Note: Taproot/Taptweak actualy uses tagged hashes (See Below).
tag = "TapTweak"
ss = hashlib.sha256(tag.encode('utf-8')).digest()
ss += ss
ss += hashlib.sha256(b'commitment').digest()
t = hashlib.sha256(ss).digest()

# 3) 
tweak_private = ECKey()
tweak_private.set(tweak, True)
tweak_point = tweak_private.get_pubkey()

# 4) 
privatekey_tweaked = privatekey.add(tweak)

# 5)
publickey_tweaked = publickey.add(tweak_point)

# 6)
msg = hashlib.sha256(b'msg').digest()
sig = privatekey_tweaked.sign_schnorr(msg)
print(publickey_tweaked.verify_schnorr(sig, msg))

**Note: Tagged Hash in TapTweak**

The Taproot Proposal describes tagged hashes for both the taptree and taptweak. 

The TapTweak uses this double nested hashing function because it already used in TapBranches and TapLeafs, where it provides context uniqueness across the Bitcoin protocol and the 64B length lends itself to optimization in implementations. 

* **`t = sha256(sha256("TapTweak") + sha256("TapTweak") + commitment)`**

## 2.3 TapTweak Script Commitments

The TapTweak commits a Taptree to the segwit version 1 public key. It does so with a familiar merkle tree construction, where the root is taggedhashed to derive TapTweak.

However, please note that the taptree has several key improvements which solve the tree height ambiguity currently found in the transaction merkle tree, which allow an attacker to create a tree node which can be interpreted as both a leaf and internal node.

![test](images/taptweak2.jpg)

**The TapTree is different than the header merkle tree in the following ways:**

* Tapleafs can be located at different heights.
* Ordering of TapLeafs is determined lexicograpically.
* Location of nodes are tagged (No ambiguity of node height).
    * Taggedhashes of internal nodes have the tag `"TapBranch"`.
    * Taggedhashes of leaf nodes have the tag `"TapLeaf"`.
    * Taggedhash of the merkle root has the tag `"TapTweak"`.
 
![test](images/taptweak3.jpg)


### Example 2.3.1: Constructing a TapTweak from TapScripts.

In the cell below, we will commit three pay-to-pubkey scripts to a taptweak and then derive the segwit address which can be spent by fulfilling these scriptpaths and the internal. We will use the same merkle tree structure as in the previous illustration.

* 1. Compute TapLeafs A, B and C.
* 2. Compute Internal node TapBranch AB.
* 3. Compute TapTweak
* 4. Derive the segwit output address.

In [ ]:
from test_framework.key import ECKey, ECPubKey
from test_framework.script import TapLeaf
from test_framework.messages import ser_string
from test_framework.address import program_to_witness
import hashlib

TAPSCRIPT_VER = bytes([0xc0]) # See TapScript chapter for more details.
internal_pubkey = ECPubKey()
internal_pubkey.set(bytes.fromhex('03af455f4989d122e9185f8c351dbaecd13adca3eef8a9d38ef8ffed6867e342e3'))

# Pay-to-Pubkey Output Scripts. See TapScript chapter for more details.
scriptA = bytes.fromhex('2103ae2011ef6f77373794a9e962c883228a6b4c0ece914a44bf43175cf054992255ac')
scriptB = bytes.fromhex('21020cd4f4c7eda281ef2e9d383e987365c843a9a5f55b49ea4252378d9430596563ac')
scriptC = bytes.fromhex('21039f10b25a8d0940abda464934c374c3a02c3355a770bffef0d2d380e0cddb045dac')

def tagged_hash(tag, input_data):
    data = hashlib.sha256(tag.encode('utf-8')).digest()
    data += data
    data += input_data
    return hashlib.sha256(data).digest()

def tapbranch(taggedhash_left, taggedhash_right):
    if taggedhash_left > taggedhash_right:
        taggedhash_left, taggedhash_right = taggedhash_right, taggedhash_left
    return tagged_hash("TapBranch", taggedhash_left + taggedhash_right)  


# 1) Compute TapLeafs A, B and C.
# Note: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)
hash_inputC = TAPSCRIPT_VER + ser_string(scriptC)
taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)
taggedhash_leafC = tagged_hash("TapLeaf", hash_inputC)

# 2) Compute Internal node TapBranch AB.
internal_nodeAB = tapbranch(taggedhash_leafA, taggedhash_leafB)

# 3) Compute TapTweak.
rootABC = tapbranch(internal_nodeAB, taggedhash_leafC)
taptweak = tagged_hash("TapTweak", internal_pubkey.get_bytes() + rootABC)
print("TapTweak:", taptweak.hex())

# 4) Derive the segwit output address.
taproot_pubkey_b = internal_pubkey.tweak_add(taptweak).get_bytes()
taproot_pubkey_v1 = bytes([taproot_pubkey_b[0] & 1]) + taproot_pubkey_b[1:]
segwit_address = program_to_witness(1, taproot_pubkey_v1)
print('Segwit address:', segwit_address)


## 2.4 Spending a Tweaked Pubkey Output (Taproot Key Path Spend)

Now that we have understood how public keys can be tweaked with tweaks, which commit locking scripts to the Segwit version 1 output, it is important to note that one can still choose to spend the output without revealing any commitments, just like in the previous chapter. To do so, we simply tweak the signing key with our TapTweak.

* 1. Construct taproot output with tweaked public key.
* 2. Send funds from the wallet to the taproot output (Segwit Address).
* 3. Create and sign transaction which sends funds back to wallet with the tweaked private key.


**Start TestNodes**

In [ ]:
# Start TestNodes.
from test_framework.test_wrapper import TestWrapper
test = TestWrapper()
test.setup()

**Generate Wallet Balance**

In [ ]:
# Generate Coins for Bitcoin Node Wallet.
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()
print(balance)

**1) Construct taproot output with tweaked public key.**

In [ ]:
# Create Tweaked Pubkey Output.
from test_framework.address import program_to_witness
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness
from test_framework.script import  CScript, OP_1, TaprootSignatureHash
from test_framework.key import ECKey, ECPubKey
from io import BytesIO

sec = ECKey()
sec.generate()
internal_pubkey = sec.get_pubkey()

# Taptweak from example 2.3.1
taptweak = bytes.fromhex('2bb739ca44084c3c12479fdab02549884b2c950065894206bfb7bc61e17d8ad7')
taproot_pubkey = internal_pubkey.tweak_add(taptweak) 
taproot_pubkey_b = taproot_pubkey.get_bytes()

taproot_pubkey_v1 = bytes([taproot_pubkey_b[0] & 1]) + taproot_pubkey_b[1:]
segwit_address = program_to_witness(1, taproot_pubkey_v1)
print("Segwit Address:", segwit_address)

**2) Send funds from the wallet to the taproot output (Segwit Address).**

In [ ]:
# Send funds to taproot output.
txid = test.nodes[0].sendtoaddress(segwit_address, balance / 100000)
print("Funding tx:", txid)

# Deserialize wallet transaction.
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# Determine Output Index of Segwit V1 Output.
# (Wallet places change output at a random txout index.)
index = 0
outputs = tx.vout
output = outputs[index]
while (output.scriptPubKey != CScript([OP_1, taproot_pubkey_v1])):
    index += 1
    output = outputs[index]
output_value = output.nValue

**3) Spend taproot output with key path.**

In [ ]:
tx_schnorr = CTransaction()
tx_schnorr.nVersion = 1
tx_schnorr.nLockTime = 0
outpoint = COutPoint(tx.sha256, index)
tx_schnorr_in = CTxIn(outpoint = outpoint)
tx_schnorr.vin = [tx_schnorr_in]

# Generate new Bitcoin Core wallet address to send funds back to.
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])

# Determine minimum fee required for mempool acceptance.
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

# Complete output which returns funds to Bitcoin Core wallet.
dest_output= CTxOut(nValue=output_value-min_fee, scriptPubKey=scriptpubkey)
tx_schnorr.vout = [dest_output]

# Sign transaction with tweaked public key.
hash_types = [0,1,2,3,0x81,0x82,0x83]
sighash = TaprootSignatureHash(tx_schnorr, [output], hash_types[0])
tweaked_sec = sec.add(taptweak)
sig = tweaked_sec.sign_schnorr(sighash)

# Construct transaction witness.
witness = CScriptWitness()
witness.stack.append(sig)
witness_in = CTxInWitness()
witness_in.scriptWitness = witness
tx_schnorr.wit.vtxinwit.append(witness_in)

# Serialize Schnorr transaction for broadcast.
tx_schnorr_str = tx_schnorr.serialize().hex()

# Test mempool acceptance.
print(test.nodes[0].testmempoolaccept([tx_schnorr_str]))


**Shutdown TestNodes**

In [ ]:
# Shutdown TestNodes.
test.shutdown()